In [1]:
%matplotlib inline
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import logm

In [2]:
class HarmonicOscillator:
    def __init__(self, N, m, T, D, k, kB, fs, dt, g, w0, w, tau ):
        self.N   =  N           # # of data points
        self.m   =  m           # mass
        self.T   =  T           # temperature
        self.D   =  D           # diffusion constant
        self.k   =  k           # stiffness
        self.kB  =  kB          # Boltzmann constant
        self.fs  =  fs          # sampling frequency
        self.dt  =  dt          # step size
        self.g   =  g           # gamma
        self.w0  =  w0          # natural frequency
        self.w   =  w           # frequency
        self.tau =  tau         # relaxation time

        
    def calcSigma(self):
        kmw  = self.k/(self.m*self.w*self.w);   
        dtbt = -self.dt/self.tau; ee=np.exp(dtbt);  
        dd   = D/(self.w*self.w*self.tau*self.tau);  
        tt   = self.w*self.dt;   cc  = np.cos(tt);  ss=np.sin(tt)
                
        s1 = (self.kB*self.T/self.k)*(1-ee*(kmw*ss*ss+(cc+ss/(2*self.w*self.tau))**2))
        s2 = dd*ee*ss*ss
        s3 = (self.kB*self.T/self.m)*(1-ee*(kmw*ss*ss+(cc-ss/(2*self.w*self.tau))**2)) 
        return s1, s3, s2


    def calcLambda(self):
        ii  = np.eye(2)
        ll = np.asanyarray([[0, -1], [self.k/self.m, self.g/self.m]])
        ee = np.exp(-self.dt/(2*self.tau)); wt2=2*self.w*self.tau
        cc=np.cos(self.w*dt); ss=np.sin(self.w*dt) 

        Lambda = ee*((cc+ss/wt2)*ii - ll*ss/self.w ) 
        return np.real(Lambda)


    def calcXV(self, Lambda, Sigma):
        x = np.zeros([N,1])
        v = np.zeros([N,1])

        s1, s3, s2 = Sigma

        for j in np.arange(0,N-1):
            oldvec = np.array([x[j],v[j]])
            randgauss = np.random.randn(2,1)
            delx = np.sqrt(s1)*randgauss[0]
            delv = (s2/(np.sqrt(s1)))*randgauss[0]+(np.sqrt(s3 - ((s2**2)/(s1))))*randgauss[1]
            delvec = np.array([delx,delv])
            updatevec = np.dot(Lambda,oldvec)+delvec
            x[j+1] = updatevec[0]
            v[j+1] = updatevec[1]
        return x,v

In [3]:
#parameters in SI units
N   = 2**18            
m   = 2e-12             
T   = 300               
k   = 300e-6            
gf  = 30    
fs  = 2**14         
dt  = 1/fs
gc  = np.sqrt(4*m*k)
gg  = gc/gf #
w0  = np.sqrt(k/m)
tau = m/gg
w   = np.sqrt((w0**2)-(1/(4*tau*tau)))
kB  = 1.38064881313131e-23  
D   = kB*T/gg

HO = HarmonicOscillator(N, m, T, D, k, kB, fs, dt, gg, w0, w, tau)
Lambda = HO.calcLambda()
Sigma  = HO.calcSigma()
x, v   = HO.calcXV(Lambda, Sigma)

## Bayes I

In [4]:
# matrix sufficient statistics
T1_11 = np.sum(x[1:]**2)
T1_12 = np.sum(x[1:]*v[1:])
T1_21 = T1_12
T1_22 = np.sum(v[1:]**2)

T2_11 = np.sum(x[1:]*x[:-1])
T2_12 = np.sum(x[1:]*v[:-1])
T2_21 = np.sum(v[1:]*x[:-1])
T2_22 = np.sum(v[1:]*v[:-1])

T3_11 = np.sum(x[:-1]*x[:-1])
T3_12 = np.sum(x[:-1]*v[:-1])
T3_21 = T3_12
T3_22 = np.sum(v[:-1]*v[:-1])

T1 = np.matrix([[T1_11,T1_12],[T1_21,T1_22]])
T2 = np.matrix([[T2_11,T2_12],[T2_21,T2_22]])
T3 = np.matrix([[T3_11,T3_12],[T3_21,T3_22]])


invT3     = np.linalg.inv(T3)
LambdaMAP = np.dot(T2, invT3)
SigmaMAP  = (1/N)*( T1 - np.dot( T2, np.dot(invT3, T2.transpose()) ) )

L_t  = LambdaMAP.transpose()
eps  = np.asanyarray([[1,0],[0,-1]])

L_te = np.dot(eps, L_t)
Lte2 = np.dot(L_te, L_te)

II   = np.eye(2)
cc   = np.linalg.inv(II - Lte2)
cMAP = np.dot(SigmaMAP, cc)


#MAP estimate of k and m and gamma
ll = -logm(LambdaMAP)/dt
m_MAP = kB*T/cMAP[1,1]
k_MAP = kB*T/cMAP[0,0]
g_MAP = m*ll[1,1]

## Bayes II

In [5]:
#MAP estimate of c using Bayes II
c_B2 = T3/N

k_B2 = kB*T/c_B2[0,0]
m_B2 = kB*T/c_B2[1,1]


In [6]:
print '\t\t simulation\t Bayes I\t Bayes II'
print 
print ('mass      \t %2.4e\t  %2.4e\t %2.4e\t'%(m, m_MAP, m_B2))
print ('stiffness \t %2.4e\t  %2.4e\t %2.4e\t'%(k, k_MAP, k_B2))
print ('friction  \t %2.4e\t  %2.4e\t'%(       gg, g_MAP))

		 simulation	 Bayes I	 Bayes II

mass      	 2.0000e-12	  1.9763e-12	 1.9704e-12	
stiffness 	 3.0000e-04	  3.0836e-04	 2.9572e-04	
friction  	 1.6330e-09	  1.6012e-09	
